## EE232E Project 2
### Problem 1

In [ ]:
from igraph import *
from collections import defaultdict

In [ ]:
outfile = open("./project_2_data/combined.txt", "w")
threshold = 10
with open("./project_2_data/actor_movies.txt", "rb") as infile:
    for line in infile:
        arr = line.split("\t\t")
        if len(arr) > threshold:
            outfile.write(line)
with open("./project_2_data/actress_movies.txt", "rb") as infile:
    for line in infile:
        arr = line.strip().split("\t\t")
        if len(arr) > threshold:
            outfile.write(line)
outfile.close()

### Problem 2
Construct a weighted directed graph $G(V, E)$ from the list, while:

$V = \text{all actors/actresses in the list.} $

$S_i = \{m | i \in V, m \text{ is a movie in which } i \text{ has acted.}$

$ E = \{(i,j)|i,j ∈ V,S_i \cap S_j \neq ∅\} \text{ and for each directed Edge } i → j, \text{ a weight is assigned as } \frac{|S_i \cap S_j|}{|S_i|}.$

In [ ]:
actor2movie = defaultdict(list)
movie2actor = defaultdict(list)

with open("./project_2_data/combined.txt", "r") as infile:
    for line in infile:
        arr = line.strip().translate(None, "&$ ").split("\t\t")
        for i in range(1, len(arr)):
            actor2movie[arr[0]].append(arr[i])
            movie2actor[arr[i]].append(arr[0])


In [ ]:
import pickle
# use pickle to save the actor2movie dict and movie2actor dict
# pickle.dump(actor2movie, open("actor2movie.pkl", "wb" ))
# pickle.dump(movie2actor, open("movie2actor.pkl", "wb" ))

# uncomment this to load pickle file
actor2movie = pickle.load(open("actor2movie.pkl", "rb" ))
movie2actor = pickle.load(open("movie2actor.pkl", "rb" ))



In [ ]:
# create directed graph
g = Graph(directed=True)
# add vertices
for i, movies in actor2movie.iteritems():
    g.add_vertex(i)

In [ ]:
# create edges
# WARNING: this step takes extremely long time, have not finished yet.

for i, movies in actor2movie.iteritems():
    # for an actor i, get the list of actors that appear in the same movie
    i2k = defaultdict(int)
    for j in movies:
        for k in movie2actor[j]:
            i2k[k]=i2k[k]+1
    for k, w in i2k.iteritems():
        weight = w / len(movies)
        g.add_edge(i, k, weight=weight)

In [ ]:
print(g.vcount())
print(g.ecount())

In [ ]:
# igraph example

# e = Graph(directed=True)
# e.add_vertex("nick")
# e.add_vertex("lisa")
# e.add_vertex("jennie")
# e.add_edge("nick", "lisa", weight=1)
# e.add_edge("jennie", "nick", weight=30)
# visual_style = {}
# visual_style["vertex_size"] = 20
# # visual_style["vertex_color"] = [color_dict[gender] for gender in g.vs["gender"]]
# visual_style["vertex_label"] = e.vs["name"]
# visual_style["edge_width"] = e.es["weight"]
# visual_style["edge_length"] = e.es["weight"]
# # visual_style["layout"] = layout
# visual_style["bbox"] = (300, 300)
# visual_style["margin"] = 20
# plot(e, **visual_style)